In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

In [2]:
np.random.seed(200831)

In [3]:
appliances = pd.read_csv("../../../datasets/energydata_complete.csv")

In [4]:
appliances.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
appliances.rename(columns={column: column.lower() for column in appliances.columns.values}, 
                 inplace=True)

In [6]:
appliances.head()

,date,appliances,lights,t1,rh_1,t2,rh_2,t3,rh_3,t4,...,t9,rh_9,t_out,press_mm_hg,rh_out,windspeed,visibility,tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [7]:
appliances.shape

(19735, 29)

In [8]:
appliances.date.min()

'2016-01-11 17:00:00'

In [9]:
appliances.date.max()

'2016-05-27 18:00:00'

1. Transforma la columna de fecha -date- en 3 nuevas columnas para tener el mes, el día del mes, y la hora del día separados.

In [10]:
appliances['year'] = pd.DatetimeIndex(appliances.date).year
appliances['month'] = pd.DatetimeIndex(appliances.date).month
appliances['day'] = pd.DatetimeIndex(appliances.date).day
appliances['hour'] = pd.DatetimeIndex(appliances.date).hour

In [11]:
appliances.columns.values

array(['date', 'appliances', 'lights', 't1', 'rh_1', 't2', 'rh_2', 't3',
       'rh_3', 't4', 'rh_4', 't5', 'rh_5', 't6', 'rh_6', 't7', 'rh_7',
       't8', 'rh_8', 't9', 'rh_9', 't_out', 'press_mm_hg', 'rh_out',
       'windspeed', 'visibility', 'tdewpoint', 'rv1', 'rv2', 'year',
       'month', 'day', 'hour'], dtype=object)

In [12]:
appliances.groupby(['month'])['year'].count()

month
1    2922
2    4176
3    4464
4    4320
5    3853
Name: year, dtype: int64

2. ¿Cuántas observaciones tienes en el conjunto de entrenamiento y cuántas en el de pruebas?

In [59]:
train = appliances[appliances.date <= '2016-04-27 23:55']
test = appliances[appliances.date >= '2016-04-28 23:55']

In [60]:
print("train: ", train.shape[0], "\ntest: ", test.shape[0])

train:  15450 
test:  4141


In [15]:
(train.date.min(), train.date.max())

('2016-01-11 17:00:00', '2016-04-27 23:50:00')

In [16]:
(test.date.min(), test.date.max())

('2016-04-29 00:00:00', '2016-05-27 18:00:00')

In [17]:
train[train.date >= '2016-04-27 00:00']

,date,appliances,lights,t1,rh_1,t2,rh_2,t3,rh_3,t4,...,rh_out,windspeed,visibility,tdewpoint,rv1,rv2,year,month,day,hour
15306,2016-04-27 00:00:00,60,0,21.500000,36.00,18.760000,37.656667,22.050000,35.950000,20.000000,...,81.000000,6.000000,40.000000,0.100000,25.942795,25.942795,2016,4,27,0
15307,2016-04-27 00:10:00,60,0,21.500000,36.00,18.700000,37.817500,22.100000,35.900000,20.000000,...,81.833333,5.833333,40.000000,0.183333,2.282755,2.282755,2016,4,27,0
15308,2016-04-27 00:20:00,60,0,21.426667,35.86,18.633333,37.900000,22.100000,35.900000,20.000000,...,82.666667,5.666667,40.000000,0.266667,16.667259,16.667259,2016,4,27,0
15309,2016-04-27 00:30:00,50,0,21.390000,35.79,18.600000,37.933333,22.100000,35.966667,20.000000,...,83.500000,5.500000,40.000000,0.350000,13.784903,13.784903,2016,4,27,0
15310,2016-04-27 00:40:00,50,0,21.323333,35.79,18.533333,38.000000,22.100000,35.900000,20.000000,...,84.333333,5.333333,40.000000,0.433333,9.383915,9.383915,2016,4,27,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15445,2016-04-27 23:10:00,70,20,21.730000,35.29,19.000000,37.433333,21.290000,34.760000,20.566667,...,88.833333,1.000000,37.833333,0.616667,43.464323,43.464323,2016,4,27,23
15446,2016-04-27 23:20:00,70,10,21.700000,35.53,18.890000,37.500000,21.323333,34.790000,20.566667,...,89.666667,1.000000,35.666667,0.533333,30.317104,30.317104,2016,4,27,23
15447,2016-04-27 23:30:00,70,20,21.700000,35.53,18.823333,37.433333,21.390000,34.863333,20.566667,...,90.500000,1.000000,33.500000,0.450000,49.482761,49.482761,2016,4,27,23
15448,2016-04-27 23:40:00,60,10,21.600000,35.40,18.760000,37.400000,21.390000,34.933333,20.566667,...,91.333333,1.000000,31.333333,0.366667,29.941927,29.941927,2016,4,27,23


In [18]:
test.head()

,date,appliances,lights,t1,rh_1,t2,rh_2,t3,rh_3,t4,...,rh_out,windspeed,visibility,tdewpoint,rv1,rv2,year,month,day,hour
15594,2016-04-29 00:00:00,30,0,20.666667,33.790000,18.066667,36.060000,22.500000,35.200000,20.000000,...,79.666667,3.333333,40.0,0.033333,19.181156,19.181156,2016,4,29,0
15595,2016-04-29 00:10:00,20,0,20.600000,33.790000,18.000000,36.060000,22.500000,35.200000,19.963333,...,79.777778,3.388889,40.0,0.088889,41.192372,41.192372,2016,4,29,0
15596,2016-04-29 00:20:00,40,0,20.600000,33.900000,18.000000,36.200000,22.390000,35.126667,19.963333,...,79.888889,3.444444,40.0,0.144444,23.867322,23.867322,2016,4,29,0
15597,2016-04-29 00:30:00,50,0,20.600000,33.900000,17.926667,36.333333,22.390000,35.126667,19.890000,...,80.000000,3.500000,40.0,0.200000,33.345619,33.345619,2016,4,29,0
15598,2016-04-29 00:40:00,50,0,20.566667,33.933333,17.890000,36.400000,22.323333,35.200000,19.890000,...,80.111111,3.555556,40.0,0.255556,9.737663,9.737663,2016,4,29,0


3. Selección de *features*

In [19]:
train_x = train.drop(['date', 'appliances', 'rv1', 'rv2', 'year'], axis=1)
test_x = test.drop(['date', 'appliances', 'rv1', 'rv2', 'year'], axis=1)

train_y = train.appliances
test_y = test.appliances

In [20]:
train_x.columns

Index(['lights', 't1', 'rh_1', 't2', 'rh_2', 't3', 'rh_3', 't4', 'rh_4', 't5',
       'rh_5', 't6', 'rh_6', 't7', 'rh_7', 't8', 'rh_8', 't9', 'rh_9', 't_out',
       'press_mm_hg', 'rh_out', 'windspeed', 'visibility', 'tdewpoint',
       'month', 'day', 'hour'],
      dtype='object')

In [21]:
train_y[:10]

0    60
1    60
2    50
3    50
4    60
5    50
6    60
7    60
8    60
9    70
Name: appliances, dtype: int64

4. GridSearchCV y TimeSeriesSplit

In [22]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [23]:
cv = TimeSeriesSplit(n_splits = 7)

In [24]:
dtr = DecisionTreeRegressor()

In [25]:
grid = {'max_depth': [5,10,15,20], 'criterion': ['mse', 'mae'], 'min_samples_leaf': [5,7,9,11]}

In [26]:
gs = GridSearchCV(dtr, param_grid=grid, cv=cv, scoring='neg_root_mean_squared_error')

In [27]:
gs.fit(train_x, train_y)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=7, test_size=None),
             estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': [5, 10, 15, 20],
                         'min_samples_leaf': [5, 7, 9, 11]},
             scoring='neg_root_mean_squared_error')

5. Mejor estimador

In [28]:
model = gs.best_estimator_
model

DecisionTreeRegressor(criterion='mae', max_depth=5, min_samples_leaf=11)

6. ¿Cuáles son las tres variables que brindan más información del mejor estimador obtenido con el GridSearchCV? Incluye el porcentaje de importancia


In [29]:
model.feature_importances_

array([2.10272075e-02, 0.00000000e+00, 0.00000000e+00, 7.95624068e-03,
       1.22185125e-02, 9.78901755e-02, 5.90324643e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.97265042e-04, 0.00000000e+00, 5.67592527e-02,
       0.00000000e+00, 1.13660581e-02, 0.00000000e+00, 0.00000000e+00,
       4.34751723e-02, 0.00000000e+00, 0.00000000e+00, 1.13660581e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.88641046e-01])

In [32]:
feature_importance = pd.DataFrame({'features': train_x.columns.values, 
                                  'importance': model.feature_importances_})

In [34]:
feature_importance.sort_values(by='importance', ascending=False).head()

,features,importance
27,hour,0.688641
5,t3,0.097890
6,rh_3,0.059032
15,t8,0.056759
20,press_mm_hg,0.043475


In [38]:
feature_importance.sort_values(by="importance").tail(6)

,features,importance
0,lights,0.021027
20,press_mm_hg,0.043475
15,t8,0.056759
6,rh_3,0.059032
5,t3,0.097890
27,hour,0.688641


7. Utilizando el mejor estimador encontrado en el GridSearchCV realiza la predicción de energía utilizada en cada hogar del conjunto de prueba.8. 

In [39]:
predictions = gs.predict(test_x)

In [53]:
predictions

array([ 50.,  50.,  50., ..., 110., 110., 110.])

8. ¿Cuál es el valor de RMSE y MAE para el conjunto de pruebas? 

In [43]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

RMSE

In [46]:
mean_squared_error(test_y, predictions,squared=False)

113.96956002976589

MAE

In [42]:
mean_absolute_error(test_y, predictions)

65.9188601787008

9. ¿Cuál es el valor del peor residual? (la más alejada al valor real) 

780

In [118]:
predictions_df = pd.DataFrame({'true_value': test_y, 'prediction': predictions, 
                               'residual': test_y - predictions})

In [119]:
predictions_df.head()

,true_value,prediction,residual
15594,30,50.0,-20.0
15595,20,50.0,-30.0
15596,40,50.0,-10.0
15597,50,50.0,0.0
15598,50,50.0,0.0


In [124]:
predictions_df.describe()

,true_value,prediction,residual
count,4141.000000,4141.000000,4141.000000
mean,96.409080,117.049022,-20.639942
std,91.626873,78.329076,112.098564
min,20.000000,50.000000,-240.000000
25%,50.000000,60.000000,-30.000000
50%,60.000000,90.000000,-10.000000
75%,100.000000,110.000000,10.000000
max,850.000000,260.000000,780.000000


10. ¿Qué valores tenían las variables predictivas correspondientes a la peor predicción?

In [125]:
predictions_df[predictions_df.residual == 780]

,true_value,prediction,residual
15798,840,60.0,780.0


In [130]:
test[test.appliances == 840]

,date,appliances,lights,t1,rh_1,t2,rh_2,t3,rh_3,t4,...,rh_out,windspeed,visibility,tdewpoint,rv1,rv2,year,month,day,hour
15798,2016-04-30 10:00:00,840,0,21.39,37.7,20.133333,38.466667,22.7,36.59,19.823333,...,83.0,1.0,40.0,4.3,47.772522,47.772522,2016,4,30,10
